# Data Science 100 Group 008-05 Project Proposal

#### Zack Peachell, Jasper Palov, Arjun Saxena, Bhavya Dubey

\
\

## Introduction to our project: Will need to choose a dataset first

## Preliminary exploratory data analysis:
\
from the dataset we have chosen!

In [7]:
# Run this cell to continue.
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.4     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [14]:
reader <- read_csv("DATA/Whistler Blackcomb - BC Canada.csv")
reader

Rows: 671 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): Date, 24 hr New Snow, Season Snowfall Total, Base Depth

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Date,24 hr New Snow,Season Snowfall Total,Base Depth
<chr>,<chr>,<chr>,<chr>
1-Jan-17,8 cm,8 cm,0 cm
6-Jan-17,1 cm,9 cm,0 cm
7-Jan-17,4 cm,13 cm,0 cm
⋮,⋮,⋮,⋮
21-Dec-09,20 cm,836 cm,241 cm
22-Dec-09,18 cm,854 cm,251 cm
30-Dec-09,3 cm,857 cm,216 cm


## Methods

## Expected outcomes and significance